#  <center> Problem Set 4 <center>

<center> 3.C01/3.C51, 7.C01/7.C51, 10.C01/10.C51, 20.C01/20.C51 <center>

In [ ]:
import os 
import glob 
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import imageio
from skimage import io, color

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms
from torch import nn 
from torchvision.models import vgg16
import torch.optim as optim

## Part 1: Classifying  Steel  Surface  Defects

In [ ]:
!wget https://raw.githubusercontent.com/vikram-sundar/ML4MolEng_Spring2022/master/psets/ps5/data/neu_surface_defect_jpg.tar.gz
!tar -xf neu_surface_defect_jpg.tar.gz

### 1.1 (15 points)  Build Image Datasets and DataLoaders

Get all the image filepaths.

In [ ]:
files = glob.glob(os.path.join('neu_surfae_defect_images', "*.jpg"))

Visualize a random image.

In [ ]:
idx = 30
img = Image.open(files[idx])
print(files[idx])
img

Your ImageDataset class.

In [ ]:
# dictionary labels 
label_dict = {
'Cr': 0, 
'In': 1, 
'Pa': 2,
'PS': 3, 
'RS': 4,
'Sc': 5
}

class ImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        
        '''
        Image dataset object that loads and transforms images. 
        
        '''
        
        self.paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        #########your implementation here######### 
        
        
        # read images given file path 
        img = # fill in
        label = # fill in
        #########your implementation here#########

        # transform images 
        if self.transform:
            img = self.transform(img)

        sample = img, label
        return sample

Transform your dataset, split the data, and define your Datasets and DataLoaders.

### 1.2 (10 points) Understand the Model Architecture

Define and load a pretrained VGG16 model.

In [ ]:
class VGG_fc1(nn.Module):
    def __init__(self, pretrain=True):
        super(VGG_fc1, self).__init__()
        self.features = vgg16(pretrained=pretrain).features # convolutional layers
        self.avgpool = vgg16(pretrained=pretrain).avgpool
        self.fc1 = vgg16(pretrained=pretrain).classifier[0] # first layer of classifier
        
    def forward(self, x):
        """Extract first fully connected feature vector"""
        # Apply convolutions
        x = self.features(x)
        # Apply pooling
        x = self.avgpool(x)
        # Flatten and convert to vectors
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x
    
model = VGG_fc1(pretrain=True).eval() # turn model into evaluation mode

The model architecture.

In [ ]:
print(model)

Choose an image from your training set and visualize 4 channels in each of layers 1, 5, and 10.

What do you observe?

### 1.3 (20 points) Train a Classifier with Transfer Learning

Define a VGG-based transfer learning classifier.

Train your classifier.

Compute and plot a test confusion matrix.

Why do you need to resize images to specific shapes and normalize pixel values to specific values for each color channel?

What are the benefits of transfer learning versus training the entire stack (CNN + MLP) again. What are the potential limitations of this approach? 

### 1.4 (15 points) Obtain Saliency Maps

Compute the saliency map for two images of each class.


Comment on any pattern you observe in the saliency maps.

# Part 2: Image Segmentation

### 2.1 (15 points) Build Datasets and DataLoaders

Download and unzip data.

In [ ]:
!wget -O part2data.zip https://raw.githubusercontent.com/vikram-sundar/ML4MolEng_Spring2022/master/psets/ps5/data/part2_data.zip
! rm -r segmentation_data
! mkdir segmentation_data/
! unzip part2data.zip -d segmentation_data

Parse data from image filepaths.

In [ ]:
paths = glob.glob("segmentation_data/part2_segdata/*")

def parse_data(path):

    # define data folders
    x_path = os.path.join(path, "images/")
    y_path = os.path.join(path, "masks/")

    # get all data paths 
    x_file = glob.glob(os.path.join(x_path, "*.png"))[0]
    y_files = glob.glob(os.path.join(y_path, "*.png"))

    # parse in data
    x = imageio.imread(x_file)
    
    masks = np.array([imageio.imread(y) for y in y_files])
    y = np.zeros_like(masks[0])
    for y_raw in masks:
        y = np.maximum(y, y_raw)

    # normalize
    x = x / 255.0
    y = y / 255.0

    return x, y

Load one image.

In [ ]:
x, y = parse_data(paths[0])
print(x.shape, y.shape)

Your ImageDataset class.

Split your data and load your DataLoaders.

Is it necessary to apply random translation to your image?  Briefly justify your answer.

### 2.2 (20 points) Train a U-Net Model that Performs Image Segmentation

Implement Dice loss.

In [ ]:
def dice_loss(pred, target):
    """Calculate Dice loss.

    Parameters
    ----------
        pred:
            predictions from the model
        target:
            ground truth label
    """

The U-Net Model.

In [ ]:
class DownSampling(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, name=None):
        super(DownSampling, self).__init__()

        self.conv = ConvBlock(in_channels, out_channels, kernel_size)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):

        conv_out = self.conv(x)
        output = self.max_pool(conv_out)

        return output, conv_out


class UpSampling(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, name=None):
        super(UpSampling, self).__init__()

        self.conv = ConvBlock(in_channels, out_channels, kernel_size)
        self.conv_t = nn.ConvTranspose2d(out_channels, out_channels, kernel_size, \
                                         padding=1, stride=2, output_padding=1)


    def forward(self, x, skip):

        conv_out = self.conv(x)
        output = self.conv_t(conv_out)

        output += skip

        return output


class ConvBlock(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding=1, stride=1, name=None):
        super(ConvBlock, self).__init__()

        block = []
        # first conv layer
        block.append(nn.Conv2d(in_channels, out_channels, kernel_size, \
                               padding=padding, stride=stride))
        block.append(nn.ReLU())
        block.append(nn.BatchNorm2d(out_channels))

        # second conv layer
        block.append(nn.Conv2d(out_channels, out_channels, kernel_size, \
                               padding=padding, stride=stride))
        block.append(nn.ReLU())
        block.append(nn.BatchNorm2d(out_channels))

        # make sequential
        self.conv_block = nn.Sequential(*block)


    def forward(self, x):

        output = self.conv_block(x)

        return output
    
    
class UNet(nn.Module):

    def __init__(self, num_kernel=8, kernel_size=3, dim=4, target_dim=1):
        """UNet

        Inputs:
            num_kernel: int
                number of kernels to use for the first layer
            kernel_size: int
                size of the kernel for the first layer
            dims: int
                number of color channels for input images 
            target_dim: int 
                number of channels for the output mask
        """

        super(UNet, self).__init__()

        self.num_kernel = num_kernel
        self.kernel_size = kernel_size
        self.dim = dim
        self.target_dim = 1

        # encode
        self.encode_1 = DownSampling(self.dim, num_kernel, kernel_size)
        self.encode_2 = DownSampling(num_kernel, num_kernel*2, kernel_size)
        self.encode_3 = DownSampling(num_kernel*2, num_kernel*4, kernel_size)
        self.encode_4 = DownSampling(num_kernel*4, num_kernel*8, kernel_size)

        # bridge
        self.bridge = nn.Conv2d(num_kernel*8, num_kernel*16, kernel_size, padding=1, stride=1)

        # decode
        self.decode_4 = UpSampling(num_kernel*16, num_kernel*8, kernel_size)
        self.decode_3 = UpSampling(num_kernel*8, num_kernel*4, kernel_size)
        self.decode_2 = UpSampling(num_kernel*4, num_kernel*2, kernel_size)
        self.decode_1 = UpSampling(num_kernel*2, num_kernel, kernel_size)

        self.segment = nn.Conv2d(num_kernel, self.target_dim, 1, padding=0, stride=1)
        self.activate = nn.Sigmoid()


    def forward(self, x):

        x, skip_1 = self.encode_1(x)
        x, skip_2 = self.encode_2(x)
        x, skip_3 = self.encode_3(x)
        x, skip_4 = self.encode_4(x)

        x = self.bridge(x)

        x = self.decode_4(x, skip_4)
        x = self.decode_3(x, skip_3)
        x = self.decode_2(x, skip_2)
        x = self.decode_1(x, skip_1)

        x = self.segment(x)

        pred = self.activate(x)

        return pred


    def args_dict(self):
        """model arguments to be saved
        """

        model_args = {'dim': self.dim,
                      'target_dim': self.target_dim,
                      'num_kernel' : self.num_kernel,
                      'kernel_size' : self.kernel_size}

        return model_args


Example model usage.

In [ ]:
model = UNet()
y = model(torch.randn(1, 4, 256, 256))

print(y.shape)

A function to plot a segmentation map.

In [ ]:
def plot_seg(image, pred_seg, true_seg, mask_cutoff=0.1):

    """ Visualize segmentation results.
    Inputs:
        image: orginal image, shape: 256 x 256 x 4
        pred_seg: predicted mask, shape: 256 x 256 x 1 or 256 x 256 
        true_seg: true mask, shape: 256 x 256 x 1 or 256 x 256
        mask_cutoff: if the mask values is larger than mask_cutoff, the mask will appear on the image
    """
    
    fig, ax = plt.subplots(1, 3, sharex='col', sharey='row')
    fig.set_size_inches((15,15))
    
    pred_seg = pred_seg.squeeze()
    true_seg = true_seg.squeeze()

    ax[0].set_title("Original Image")
    ax[1].set_title("Prediction")
    ax[2].set_title("Ground Truth")

    pred_mask = np.zeros((256, 256))
    pred_mask[pred_seg > mask_cutoff] = 1
    y_mask = np.zeros((256, 256))
    y_mask[true_seg > mask_cutoff] = 1

    ax[0].imshow(image)
    ax[1].imshow(color.label2rgb(pred_mask,image,colors=[(255,0,0)],alpha=0.0025, bg_label=0, bg_color=None))
    ax[2].imshow(color.label2rgb(y_mask,image,colors=[(255,0,0)],alpha=0.0025, bg_label=0, bg_color=None))

    plt.show()

Train your model.

Show segmentation results for 3 images from the test dataset.